<a href="https://colab.research.google.com/github/IndioP/Projeto_Ciencia_Dados/blob/main/preprocessamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

##Acidentes

In [ ]:
acidentes=pd.read_csv("/content/drive/MyDrive/ciencia-de-dados/dataset preprocessado/acidentes_transito_2015_2021.csv",delimiter = ';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,25,26,27,28,29,30,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
acidentes.head()

,DATA,HORA,NATUREZA_ACIDENTE,SITUACAO,BAIRRO,ENDERECO,NUMERO,DETALHE_ENDERECO_ACIDENTE,COMPLEMENTO,BAIRRO_CRUZAMENTO,NUM_SEMAFORO,SENTIDO_VIA,TIPO,DESCRICAO,AUTO,MOTO,CICLOM,CICLISTA,PEDESTRE,ONIBUS,CAMINHAO,VIATURA,OUTROS,VITIMAS,VITIMASFATAIS,ACIDENTE_VERIFICADO,TEMPO_CLIMA,SITUACAO_SEMAFORO,SINALIZACAO,CONDICAO_VIA,CONSERVACAO_VIA,PONTO_CONTROLE,SITUACAO_PLACA,VELOCIDADE_MAX_VIA,MAO_DIRECAO,DIVISAO_VIA1,DIVISAO_VIA2,DIVISAO_VIA3,NATUREZA,ENDERECO_CRUZAMENTO,NUMERO_CRUZAMENTO,REFERENCIA_CRUZAMENTO
0,2015-12-03,11:36:00.000,SEM VÍTIMA,CANCELADA,CASA AMARELA,AV NORTE,NaN,NaN,SENTIDO SUBURBIO E/F AO SESC,NaN,NaN,NaN,COLISÃO,COLISÃO SEM VITIMAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-06-01,06:35:00.000,SEM VÍTIMA,FINALIZADA,ENCRUZILHADA,EST DE BELEM,NaN,NaN,IGREJA BATISTA MEMORIAL DE BELÉM,NaN,NaN,NaN,COLISÃO,SEM VÍTIMAS.,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-06-01,08:04:00.000,SEM VÍTIMA,FINALIZADA,BOA VIAGEM,RUA ERNESTO DE PAULA SANTOS,NaN,NaN,SEMAFORO Nº217,NaN,NaN,NaN,COLISÃO,SEM VITIMA,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-01,08:06:00.000,SEM VÍTIMA,FINALIZADA,AFOGADOS,AV SUL,NaN,NaN,AO LADO DA ESTAÇAO LARGO DA PAZ,NaN,NaN,NaN,COLISÃO,COLISAO S/V,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-06-01,08:13:00.000,SEM VÍTIMA,CANCELADA,MADALENA,RUA BENFICA,NaN,NaN,EM FRENTE AO CLUBE INTERNACIONAL,NaN,NaN,NaN,COLISÃO,GM RIBEIRO JA NO LOCAL VT 05,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
acidentes.dtypes

DATA                          object
HORA                          object
NATUREZA_ACIDENTE             object
SITUACAO                      object
BAIRRO                        object
ENDERECO                      object
NUMERO                        object
DETALHE_ENDERECO_ACIDENTE     object
COMPLEMENTO                   object
BAIRRO_CRUZAMENTO             object
NUM_SEMAFORO                 float64
SENTIDO_VIA                   object
TIPO                          object
DESCRICAO                     object
AUTO                         float64
MOTO                         float64
CICLOM                       float64
CICLISTA                     float64
PEDESTRE                     float64
ONIBUS                       float64
CAMINHAO                     float64
VIATURA                      float64
OUTROS                       float64
VITIMAS                      float64
VITIMASFATAIS                float64
ACIDENTE_VERIFICADO           object
TEMPO_CLIMA                   object
S

In [ ]:
#ajustando tipos dos dados

acidentes['DATA'] = pd.to_datetime(acidentes['DATA'], format='%Y-%m-%d', errors='ignore')
acidentes['HORA'] = pd.to_datetime(acidentes['HORA'], format='%H:%M:%S', errors = 'ignore')

acidentes = acidentes.astype({'NATUREZA_ACIDENTE':'category','SITUACAO':'category','BAIRRO':'category','ENDERECO':'category','NUMERO':'category','DETALHE_ENDERECO_ACIDENTE':'category','COMPLEMENTO':'category','BAIRRO_CRUZAMENTO':'category','NUM_SEMAFORO':'category','SENTIDO_VIA':'category','TIPO':'category','DESCRICAO':'category','AUTO':'float','MOTO':'float','CICLOM':'float','CICLISTA':'float','PEDESTRE':'float','ONIBUS':'float','CAMINHAO':'float','VIATURA':'float','OUTROS':'float','VITIMAS':'float','VITIMASFATAIS':'float','ACIDENTE_VERIFICADO':'category','TEMPO_CLIMA':'category','SITUACAO_SEMAFORO':'category','SINALIZACAO':'category','CONDICAO_VIA':'category','CONSERVACAO_VIA':'category','PONTO_CONTROLE':'category','SITUACAO_PLACA':'category','VELOCIDADE_MAX_VIA':'category','MAO_DIRECAO':'category','DIVISAO_VIA1':'category','DIVISAO_VIA2':'category','DIVISAO_VIA3':'category','NATUREZA':'category','ENDERECO_CRUZAMENTO':'category','NUMERO_CRUZAMENTO':'category','REFERENCIA_CRUZAMENTO':'category'})

In [ ]:
acidentes.dtypes

DATA                         datetime64[ns]
HORA                                 object
NATUREZA_ACIDENTE                  category
SITUACAO                           category
BAIRRO                             category
ENDERECO                           category
NUMERO                             category
DETALHE_ENDERECO_ACIDENTE          category
COMPLEMENTO                        category
BAIRRO_CRUZAMENTO                  category
NUM_SEMAFORO                       category
SENTIDO_VIA                        category
TIPO                               category
DESCRICAO                          category
AUTO                                float64
MOTO                                float64
CICLOM                              float64
CICLISTA                            float64
PEDESTRE                            float64
ONIBUS                              float64
CAMINHAO                            float64
VIATURA                             float64
OUTROS                          

In [ ]:
#Tentando gerar a coluna de Dia da semana...
#não ta funcionando por enquanto....
#precisamos rever


#acidentes['DIA_DA_SEMANA'] = acidentes['DATA'].to_timestamp
#acidentes['DIA_DA_SEMANA'].unique()
#acidentes['DIA_DA_SEMANA'] = acidentes['DIA_DA_SEMANA'].day_name()


In [ ]:
print(acidentes.isnull().sum()/len(acidentes))

DATA                         0.000000
HORA                         0.000947
NATUREZA_ACIDENTE            0.007267
SITUACAO                     0.000930
BAIRRO                       0.005631
ENDERECO                     0.004064
NUMERO                       0.544842
DETALHE_ENDERECO_ACIDENTE    0.855468
COMPLEMENTO                  0.062356
BAIRRO_CRUZAMENTO            0.129051
NUM_SEMAFORO                 0.915585
SENTIDO_VIA                  0.776787
TIPO                         0.000982
DESCRICAO                    0.220561
AUTO                         0.093422
MOTO                         0.745256
CICLOM                       0.992991
CICLISTA                     0.985190
PEDESTRE                     0.978405
ONIBUS                       0.852041
CAMINHAO                     0.900499
VIATURA                      0.988307
OUTROS                       0.988531
VITIMAS                      0.004718
VITIMASFATAIS                0.997382
ACIDENTE_VERIFICADO          0.749458
TEMPO_CLIMA 

*** As colunas abaixo vão ser retiradas pela falta de valores ser muito elevada em relação ao dataset***

NUMERO além de que essa granularidade seria muito especifica para o nosso foco

DETALHE_ENDERECO_ACIDENTE é um valor muito subjetivo e pouco padronizado

NUM_SEMAFORO é um numero que não significa muita coisa pro nosso foco

SENTIDO_VIA seria muito especifico para os testes que a gente vai fazer

DIVISAO_VIA2 existem poucas vias com 3 faixas

DIVISAO_VIA3 existem poucas vias com 4 faixas

NATUREZA essa coluna ta vazia

ENDERECO_CRUZAMENTO

NUMERO_CRUZAMENTO

REFERENCIA_CRUZAMENTO


In [ ]:
excluidos = acidentes[[ 'NUMERO','DETALHE_ENDERECO_ACIDENTE','NUM_SEMAFORO','SENTIDO_VIA','DIVISAO_VIA2','DIVISAO_VIA3','NATUREZA','ENDERECO_CRUZAMENTO','NUMERO_CRUZAMENTO','REFERENCIA_CRUZAMENTO']]
excluidos.head()

,NUMERO,DETALHE_ENDERECO_ACIDENTE,NUM_SEMAFORO,SENTIDO_VIA,DIVISAO_VIA2,DIVISAO_VIA3,NATUREZA,ENDERECO_CRUZAMENTO,NUMERO_CRUZAMENTO,REFERENCIA_CRUZAMENTO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#apagando algumas tabelas que não eram tão interessantes pra gente
acidentes = acidentes.drop([ 'NUMERO','DETALHE_ENDERECO_ACIDENTE','NUM_SEMAFORO','SENTIDO_VIA','DIVISAO_VIA2','DIVISAO_VIA3','NATUREZA','ENDERECO_CRUZAMENTO','NUMERO_CRUZAMENTO','REFERENCIA_CRUZAMENTO'], axis=1)

In [ ]:
acidentes.head()

,DATA,HORA,NATUREZA_ACIDENTE,SITUACAO,BAIRRO,ENDERECO,COMPLEMENTO,BAIRRO_CRUZAMENTO,TIPO,DESCRICAO,AUTO,MOTO,CICLOM,CICLISTA,PEDESTRE,ONIBUS,CAMINHAO,VIATURA,OUTROS,VITIMAS,VITIMASFATAIS,ACIDENTE_VERIFICADO,TEMPO_CLIMA,SITUACAO_SEMAFORO,SINALIZACAO,CONDICAO_VIA,CONSERVACAO_VIA,PONTO_CONTROLE,SITUACAO_PLACA,VELOCIDADE_MAX_VIA,MAO_DIRECAO,DIVISAO_VIA1
0,2015-12-03,11:36:00.000,SEM VÍTIMA,CANCELADA,CASA AMARELA,AV NORTE,SENTIDO SUBURBIO E/F AO SESC,NaN,COLISÃO,COLISÃO SEM VITIMAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-06-01,06:35:00.000,SEM VÍTIMA,FINALIZADA,ENCRUZILHADA,EST DE BELEM,IGREJA BATISTA MEMORIAL DE BELÉM,NaN,COLISÃO,SEM VÍTIMAS.,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-06-01,08:04:00.000,SEM VÍTIMA,FINALIZADA,BOA VIAGEM,RUA ERNESTO DE PAULA SANTOS,SEMAFORO Nº217,NaN,COLISÃO,SEM VITIMA,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-01,08:06:00.000,SEM VÍTIMA,FINALIZADA,AFOGADOS,AV SUL,AO LADO DA ESTAÇAO LARGO DA PAZ,NaN,COLISÃO,COLISAO S/V,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-06-01,08:13:00.000,SEM VÍTIMA,CANCELADA,MADALENA,RUA BENFICA,EM FRENTE AO CLUBE INTERNACIONAL,NaN,COLISÃO,GM RIBEIRO JA NO LOCAL VT 05,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
acidentes['auto'] = acidentes['auto'].fillna(0)
acidentes['moto'] = acidentes['moto'].fillna(0)
acidentes['ciclom'] = acidentes['ciclom'].fillna(0)
acidentes['ciclista'] = acidentes['ciclista'].fillna(0)
acidentes['pedestre'] = acidentes['pedestre'].fillna(0)
acidentes['onibus'] = acidentes['onibus'].fillna(0)
acidentes['caminhao'] = acidentes['caminhao'].fillna(0)
acidentes['viatura'] = acidentes['viatura'].fillna(0)
acidentes['outros'] = acidentes['outros'].fillna(0)

acidentes['vitimasfatais'] = acidentes['vitimasfatais'].fillna(0)



KeyError: ignored

In [ ]:
acidentes.head()

In [ ]:
acidentes['tipo'].unique()

KeyError: ignored

In [ ]:
acidentes['descricao'].unique()

In [ ]:
acidentes['natureza_acidente'].unique()

KeyError: ignored

In [ ]:
print(acidentes.isnull().sum())

In [ ]:
acidentes[acidentes['vitimas'].isnull() && acidentes['natureza_acidente'] == 'COM VÍTIMA']

SyntaxError: ignored

In [ ]:
comercios = pd.read_csv("/content/drive/MyDrive/ciencia-de-dados/dataset preprocessado/comercios_2015_2021_trimestre.csv",delimiter = ';')

In [ ]:
comercios.head()

,ANO,TRIMESTRE,RAZAO_SOCIAL,NOME_FANTASIA,CNPJ,PORTE,DATA_INICIO_OPERACAO,SITUACAO,ATIVIDADE,CEP,UF,LOCALIDADE,BAIRRO,LOGRADOURO,DATA_GERACAO_CERTIFICADO,DATA_VALIDADE_CERTIFICADO,CODIGO_CERTIFICADO,CODIGO_DESCRICAO_CNAE,NATUREZA_JURIDICA
0,2021,1,"MELLO, RIBEIRO & MELLO LTDA",China in Box Rosarinho,00.874.830/0001-48,EMPRESA DE PEQUENO PORTE,25/10/1995,Em Operação,"Restaurante, Cafeteria, Bar e Similares",52041-305,PE,Recife,Rosarinho,Amaro Coutinho,23/03/2020 09:27,23/03/2022 09:27,00.874.830/0001-48,"5611-2/03 - LANCHONETES, CASAS DE CHÁ, DE SUCO...",Sociedade Empresária Limitada
1,2021,1,M DAS DORES DE SOUZA CHURRASCARIA,Restaurante e Churrascaria da Dodora,01.219.705/0001-67,MICROEMPRESA,28/05/1996,Em Operação,"Restaurante, Cafeteria, Bar e Similares",55690-000,PE,Barra de Guabiraba,-,Av. Eneias Texeira de Carvalho n 101,01/04/2019 10:44,01/04/2021 10:44,01.219.705/0001-67,5611-2/02 - BARES E OUTROS ESTABELECIMENTOS ES...,Empresário Individual
2,2021,1,POUSADA DO AMPARO - EIRELI,POUSADA DO AMPARO,01.790.381/0001-12,MICROEMPRESA,06/04/1997,Em Operação,"Restaurante, Cafeteria, Bar e Similares",53025-080,PE,Olinda,Amparo,do Amparo,11/06/2020 08:17,11/06/2022 08:17,01.790.381/0001-12,5611-2/01 - RESTAURANTES E SIMILARES,Empresa Individual de Responsabilidade Limitad...
3,2021,1,NEZ BISTRO COMERCIO DE BEBIDAS E RESTAURANTE E...,NEZ BISTRO,02.217.561/0001-72,MICROEMPRESA,30/10/1997,Em Operação,"Restaurante, Cafeteria, Bar e Similares",52061-420,PE,Recife,Casa Forte,de Casa Forte,16/07/2020 12:44,16/07/2022 12:44,02.217.561/0001-72,5611-2/01 - RESTAURANTES E SIMILARES,Empresa Individual de Responsabilidade Limitad...
4,2021,1,ATLANTICA HOTELS INTERNATIONAL BRASIL LTDA,RESTAURANTE NABUCO,02.223.966/0036-43,DEMAIS,03/10/2003,Em Operação,"Restaurante, Cafeteria, Bar e Similares",51111-000,PE,Recife,Boa Viagem,Boa Viagem,19/12/2019 07:58,19/12/2021 07:58,02.223.966/0036-43,5611-2/01 - RESTAURANTES E SIMILARES,Sociedade Empresária Limitada


In [ ]:
comercios.dtypes

ANO                           int64
TRIMESTRE                     int64
RAZAO_SOCIAL                 object
NOME_FANTASIA                object
CNPJ                         object
PORTE                        object
DATA_INICIO_OPERACAO         object
SITUACAO                     object
ATIVIDADE                    object
CEP                          object
UF                           object
LOCALIDADE                   object
BAIRRO                       object
LOGRADOURO                   object
DATA_GERACAO_CERTIFICADO     object
DATA_VALIDADE_CERTIFICADO    object
CODIGO_CERTIFICADO           object
CODIGO_DESCRICAO_CNAE        object
NATUREZA_JURIDICA            object
dtype: object

In [ ]:
excluidos = comercios[['ANO','TRIMESTRE','RAZAO_SOCIAL','NOME_FANTASIA','CNPJ','PORTE','SITUACAO','UF','DATA_GERACAO_CERTIFICADO','DATA_VALIDADE_CERTIFICADO','CODIGO_CERTIFICADO',	'CODIGO_DESCRICAO_CNAE',	'NATUREZA_JURIDICA']]
excluidos.head()

,ANO,TRIMESTRE,RAZAO_SOCIAL,NOME_FANTASIA,CNPJ,PORTE,SITUACAO,UF,DATA_GERACAO_CERTIFICADO,DATA_VALIDADE_CERTIFICADO,CODIGO_CERTIFICADO,CODIGO_DESCRICAO_CNAE,NATUREZA_JURIDICA
0,2021,1,"MELLO, RIBEIRO & MELLO LTDA",China in Box Rosarinho,00.874.830/0001-48,EMPRESA DE PEQUENO PORTE,Em Operação,PE,23/03/2020 09:27,23/03/2022 09:27,00.874.830/0001-48,"5611-2/03 - LANCHONETES, CASAS DE CHÁ, DE SUCO...",Sociedade Empresária Limitada
1,2021,1,M DAS DORES DE SOUZA CHURRASCARIA,Restaurante e Churrascaria da Dodora,01.219.705/0001-67,MICROEMPRESA,Em Operação,PE,01/04/2019 10:44,01/04/2021 10:44,01.219.705/0001-67,5611-2/02 - BARES E OUTROS ESTABELECIMENTOS ES...,Empresário Individual
2,2021,1,POUSADA DO AMPARO - EIRELI,POUSADA DO AMPARO,01.790.381/0001-12,MICROEMPRESA,Em Operação,PE,11/06/2020 08:17,11/06/2022 08:17,01.790.381/0001-12,5611-2/01 - RESTAURANTES E SIMILARES,Empresa Individual de Responsabilidade Limitad...
3,2021,1,NEZ BISTRO COMERCIO DE BEBIDAS E RESTAURANTE E...,NEZ BISTRO,02.217.561/0001-72,MICROEMPRESA,Em Operação,PE,16/07/2020 12:44,16/07/2022 12:44,02.217.561/0001-72,5611-2/01 - RESTAURANTES E SIMILARES,Empresa Individual de Responsabilidade Limitad...
4,2021,1,ATLANTICA HOTELS INTERNATIONAL BRASIL LTDA,RESTAURANTE NABUCO,02.223.966/0036-43,DEMAIS,Em Operação,PE,19/12/2019 07:58,19/12/2021 07:58,02.223.966/0036-43,5611-2/01 - RESTAURANTES E SIMILARES,Sociedade Empresária Limitada


In [ ]:
comercios = comercios.drop(['ANO','TRIMESTRE','RAZAO_SOCIAL','NOME_FANTASIA','CNPJ','PORTE','SITUACAO','UF','DATA_GERACAO_CERTIFICADO','DATA_VALIDADE_CERTIFICADO','CODIGO_CERTIFICADO',	'CODIGO_DESCRICAO_CNAE',	'NATUREZA_JURIDICA'], axis=1)

In [ ]:
comercios.head()

,DATA_INICIO_OPERACAO,ATIVIDADE,CEP,LOCALIDADE,BAIRRO,LOGRADOURO
0,25/10/1995,"Restaurante, Cafeteria, Bar e Similares",52041-305,Recife,Rosarinho,Amaro Coutinho
1,28/05/1996,"Restaurante, Cafeteria, Bar e Similares",55690-000,Barra de Guabiraba,-,Av. Eneias Texeira de Carvalho n 101
2,06/04/1997,"Restaurante, Cafeteria, Bar e Similares",53025-080,Olinda,Amparo,do Amparo
3,30/10/1997,"Restaurante, Cafeteria, Bar e Similares",52061-420,Recife,Casa Forte,de Casa Forte
4,03/10/2003,"Restaurante, Cafeteria, Bar e Similares",51111-000,Recife,Boa Viagem,Boa Viagem


In [ ]:
comercios['DATA_INICIO_OPERACAO'] = pd.to_datetime(comercios['DATA_INICIO_OPERACAO'], format='%Y-%m-%d', errors='ignore')
comercios = comercios.astype({'ATIVIDADE':'category',	'CEP':'category',	'LOCALIDADE':'category',	'BAIRRO':'category', 'LOGRADOURO':'category'})

**Equipamento velocidade vias**

In [ ]:
equipamento_velocidade_vias = pd.read_csv("/content/drive/MyDrive/ciencia-de-dados/dataset preprocessado/equipamento_velocidade_vias_2016_2021.csv",delimiter = ';')

In [ ]:
equipamento_velocidade_vias.head()

,ANO,EQUIPAMENTO,TIPO,LOGRADOURO,VELOCIDADE_VIA,LATITUDE,LONGITUDE
0,2016,5941,Lombada,"AV. MAL. MASCARENHAS DE MORAES, EM FRENTE AERO...",60 km/h,-8.130970,-34.916161
1,2016,5942,Lombada,"AV. MAL. MASCARENHAS DE MORAES, EM FRENTE AERO...",60 km/h,-8.130647,-34.915886
2,2016,5943,Lombada,AV. BOA VIAGEM - TERCEIRO JARDIM,60 km/h,-8.107843,-34.888161
3,2016,5944,Lombada,"DEFRONTE A PCA. GOV. PAULO GUERRA, CABANGA - S...",60 km/h,-8.080121,-34.892187
4,2016,5945,Lombada,"APOS PONTE AGAMENON MAGALHAES, CABANGA - SENT....",60 km/h,-8.080820,-34.891159


In [ ]:
excluidos = equipamento_velocidade_vias[['TIPO']]
excluidos.head()

,TIPO
0,Lombada
1,Lombada
2,Lombada
3,Lombada
4,Lombada


In [ ]:
equipamento_velocidade_vias = equipamento_velocidade_vias.drop(['TIPO'], axis=1)

In [ ]:
equipamento_velocidade_vias = equipamento_velocidade_vias.astype({'ANO':'category',	'EQUIPAMENTO':'category',	'LOGRADOURO':'category',	'VELOCIDADE_VIA':'category', 'LATITUDE':'category','LONGITUDE':'category'})

In [ ]:
equipamento_velocidade_vias.dtypes

ANO               category
EQUIPAMENTO       category
LOGRADOURO        category
VELOCIDADE_VIA    category
LATITUDE          category
LONGITUDE         category
dtype: object

**dataset de multas**

In [ ]:
multas = pd.read_csv("/content/drive/MyDrive/ciencia-de-dados/dataset preprocessado/relatorio_multas_2015_2021.csv",delimiter = ';')

NameError: ignored

In [ ]:
multas.head()

NameError: ignored

In [ ]:
excluidos = multas[['DATAIMPLANTACAO', 'AGENTEEQUIPAMENTO',	'AMPAROLEGAL']]
excluidos.head()

NameError: ignored

In [ ]:
multas = multas.drop(['DATAIMPLANTACAO', 'AGENTEEQUIPAMENTO',	'AMPAROLEGAL'], axis=1)

In [ ]:
multas['DATAINFRACAO'] = pd.to_datetime(multas['DATAINFRACAO'], format='%Y-%m-%d', errors='ignore')
multas['HORAINFRACAO'] = pd.to_datetime(multas['HORAINFRACAO'], format='%H:%M:%S', errors = 'ignore')
multas = multas.astype({'INFRACAO':'category', 'DESCRICAOINFRACAO':'category','LOCALCOMETIMENTO':'category'})

NameError: ignored

In [ ]:
multas.dtypes

**velocidades**

In [ ]:
velocidades = pd.read_csv("/content/drive/MyDrive/ciencia-de-dados/dataset preprocessado/velocidade_vias_media_15minutos_2016_2021.csv",delimiter = ';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
velocidades.head()

,ANO,MES,EQUIPAMENTO,FAIXA,DATA,HORA,MINUTOS_INTERVALO,QTD_0A10KM,QTD_11A20KM,QTD_21A30KM,QTD_31A40KM,QTD_41A50KM,QTD_51A60KM,QTD_61A70KM,QTD_71A80KM,QTD_81A90KM,QTD_91A100KM,QTD_ACIMADE100KM
0,2016,1.0,FS002REC,1.0,2016-01-01,0.0,0-15,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016,1.0,FS002REC,1.0,2016-01-01,0.0,16-30,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016,1.0,FS002REC,1.0,2016-01-01,0.0,31-45,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016,1.0,FS002REC,1.0,2016-01-01,0.0,46-59,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,1.0,FS002REC,1.0,2016-01-01,1.0,0-15,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
velocidades['DATA'] = pd.to_datetime(velocidades['DATA'], format='%Y-%m-%d', errors='ignore')

velocidades = velocidades.astype({'ANO':'category',	'MES':'category',	'EQUIPAMENTO':'category',	'FAIXA':'category',	'HORA':'category',	'MINUTOS_INTERVALO':'category',	'QTD_0A10KM':'float',	'QTD_11A20KM':'float',	'QTD_21A30KM':'float',	'QTD_31A40KM':'float',	'QTD_41A50KM':'float',	'QTD_51A60KM':'float',	'QTD_61A70KM':'float',	'QTD_71A80KM':'float',	'QTD_81A90KM':'float',	'QTD_91A100KM':'float',	'QTD_ACIMADE100KM':'float'})

#**precisa verificar se o codigo do equipamento no dataset acima é igual ao codigo do equipamento do dataset das multas**